## Pupil Segmentation CenterFix - Phase Shift

> Author: [Zhengyi Zhan](mailto:zhanzy@zju.edu.cn)
> 
> Date: Sept 15, 2024

### Introduction

This script uses a whole blazed grating on an SLM to perform pupil segmentation for wavefront sensing.

It implements a 'CenterFix' pattern where one mask is fixed in the center and another moves in a rotational order.

Key Features:
- Utilizes HPC_SLM_Calib for SLM control and GxCam for camera operations
- Generates rotational coordinates for grating mask placement
- Applies phase shifts to the moving mask
- Captures and saves image data for further analysis

### Initialize Camera & SLM

In [1]:
from hpc_slm.hpc_slm import *
from gxipy.MyGxCam import GxCam

monitor_idx = 2             # Index of the monitor connected to SLM (via DVI)
size = [1024, 1272]         # SLM resolution [height, width]

cam_model_query = '1070'    # Camera model identifier

# Initialize SLM & grating object
slm = HPC_SLM_Calib(monitor_idx, 0, size)
slm.display_img(np.zeros(size, dtype=np.uint8)) # Display a blank image on SLM to verify correct monitor

# Initialize cam
cam = GxCam(cam_model_query)
cam.set_trigger_mode(True)      # Enable trigger mode for synchronized capture

### Set Grating Cooridnates Parameters

In [2]:
center_x = 636
center_y = 460

beam_diameter = 310     # Diameter of the beam
grating_size = 29       # Size of each grating mask
d1 = 36                 # Distance from the center to the center of the first circle

grating_type = 'blazed' # The type of the grating
grating_step = 10       # Step size for the grating

turns = 5               # Number of rotational turns
# numbers = 9
numbers = np.array([9, 18, 26, 30, 40]) # Custom numbers of points for each turn

beam_diameter += grating_size   # Adjust beam diameter to include grating size

from hpc_slm.gratings import *
import matplotlib.pyplot as plt

g = Gratings(size)
# Generate rotational coordinates and preview image
cx, cy, img = g.gen_rotational_coors(grating_size, d1, beam_diameter, 
                                     center_x, center_y, turns, numbers, img_flag=True)

### Preview Rotational Cooridnates

In [ ]:
plt.figure(figsize=(9, 6))
plt.imshow(img)
plt.show()

### Acquisition

In [ ]:
from pathlib import Path

calib_file = Path(r'D:\.MINFLUX\CalibData\slm-calib\yzt-642-100ms.h5')  # SLM calibration file
flat_flag = True        # Apply flat field correction
local_flag = True       # Use local LUT for calibration
global_lut = 200        # Global LUT value (only used when local_flag is False)

slm_delay = 0.1         # Delay between SLM updates (in seconds)
frame_per_postion = 1   # Number of frames to capture per position

phase_step = 4          # Number of phase steps (use 1 to disable phase shift)

from tqdm import tqdm
import time

# Load SLM calibration file
slm.load_calib_file(calib_file)

# Generate a whole grating pattern
whole_grating = g.gen_grating(grating_type, beam_diameter,
                        grating_step = grating_step,
                        circle_mask = True, phase_inverse = False)

# Generate phase shift array
if phase_step < 1:
    phase_step = 1
phase_shift_rad = np.linspace(0, 2 * np.pi, phase_step, endpoint = False)

# Open camera stream
cam.open_stream()
time.sleep(0.1) # Short delay to ensure camera is ready

# Place the whole grating into the specified beam center of canvas
whole_grating_canvas = g.place_patterns(cx[0:1], cy[0:1], whole_grating)

# Capture image with only fixed center grating to retrieve the center of interference
grating_mask = g.gen_canvas_mask(np.array([grating_size]), cx[0:1], cy[0:1], circle_mask=True)
grating_canvas = g.apply_canvas_mask(whole_grating_canvas, grating_mask)
slm.phs_to_display(grating_canvas, flat_flag, local_flag, global_lut)
time.sleep(slm_delay)
img_center = cam.get_snapshot()

# Initialize memory and set up image array
imgs = np.zeros(img_center.shape + (phase_step, len(cx)-1,), dtype = img_center.dtype)

# Main acquisition loop
for i in tqdm(range(len(cx)-1)):
    # Set up one fixed grating in the center and one moving grating
    d_arr = np.array([grating_size, grating_size])
    cx_arr = np.array([cx[0], cx[i+1]])
    cy_arr = np.array([cy[0], cy[i+1]])

    # Generate canvas mask for two circles and apply it
    grating_mask = g.gen_canvas_mask(d_arr, cx_arr, cy_arr, circle_mask=True)
    grating_canvas = g.apply_canvas_mask(whole_grating_canvas, grating_mask)

    # Generate canvas mask for the moving circle (for phase shift only)
    phase_shift_mask = g.gen_canvas_mask(np.array([grating_size]), cx[i+1:i+2], cy[i+1:i+2], circle_mask=True)
 
    for j, value in enumerate(phase_shift_rad):
        # Apply the phase shift to the moving grating
        phase_shift_canvas = g.apply_canvas_mask(np.ones(size) * value, phase_shift_mask)

        # Send pattern to SLM
        slm.phs_to_display(grating_canvas+phase_shift_canvas, flat_flag, local_flag, global_lut)

        time.sleep(slm_delay)   # Wait for SLM to ensure the pattern is changed

        # Capture one frame from camera
        imgs[:, :, j, i] = cam.get_snapshot()

# Close camera stream
cam.close_stream()

### Data Preview

In [ ]:
# Preview the first 9 images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(imgs[:,:,0,i])
plt.show()

### Save Data

In [6]:
from scipy.io import savemat
from datetime import datetime

# Generate filename with current timestamp
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = Path(f"pupilseg_aps_{current_time}.mat")
final_path = Path(r"D:\BeamSense\PyPupilSegAPS") / file_name

# Prepare the structs of metadata
grating_para = {
    'type': grating_type,
    'size': grating_size,
    'step': grating_step,
    'beam_diameter': beam_diameter,
    'd1': d1,
    'turns': turns,
    'numbers': numbers
}

slm_para = {
    'local': local_flag,
    'flat': flat_flag,
    'calib_file': str(calib_file),
    'delay': slm_delay
}

# Save data and metadata to '.mat'
savemat(str(final_path), {'imgs': imgs, 'img_center': img_center,
                          'cx': cx, 'cy': cy, 
                          'grating_para': grating_para, 
                          'slm_para': slm_para})

### Release Camera & SLM

In [7]:
del cam
del slm